In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame as df
import numpy as np
import csv
from itertools import combinations
from sklearn.model_selection import train_test_split
import math
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import pickle

from sklearn.metrics import pairwise_distances

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_regression


In [ ]:
def getData(qual_vec_type = "500"):
    otu = pd.read_csv("data/silva/freshwater/otu_filtered_freshwater.csv", sep = "\t", index_col=0)
    print(otu.shape)
    otu.head()
    #Delete samples that have no taxa
    otu = otu.loc[:, otu.sum(axis=0) != 0]
    
    #Read quality vector data
    if qual_vec_type == "100":
        qual_vecs = pd.read_csv("embeddings/silva/glove_emb_freshwater_100.txt", sep = " ", index_col = 0, header=None)
    if qual_vec_type == "500":
        qual_vecs = pd.read_csv("embeddings/silva/glove_emb_freshwater_2perc_500.txt", sep = " ", index_col = 0, header=None)
    print(qual_vecs.shape)
    qual_vecs.head()
    
    #Match taxa present in quality vectors with those in otu table
    bools_drop = [i not in qual_vecs.index.values for i in otu.index.values]
    drop = otu.index.values[bools_drop]
    otu_drop = otu.drop(drop, axis = 0)
    print("OTU DROP SHAPE: " + str(otu_drop.shape))
    
    qual_vecs = qual_vecs.drop("<unk>", axis = 0)
    qual_vecs_sort = qual_vecs.reindex(sorted(qual_vecs.index.values), axis = 0)
    otu = otu_drop.reindex(sorted(otu_drop.index.values), axis = 0) #Organize otu rows to match taxa in qual_vecs

    ntaxa = qual_vecs.shape[0]
    print(ntaxa)
    bools_correct = [qual_vecs_sort.index.values[i] == otu.index.values[i] for i in range(ntaxa)]

    if (np.sum(bools_correct) == qual_vecs_sort.shape[0]) and (np.sum(bools_correct) == otu.shape[0]):
        print("Safe to continue")
    else:
        print("STOP! Something is wrong.")
        
        
    #Read mapping data
    mapping = pd.read_csv("data/emp_qiime_mapping_release1.tsv.csv", sep = ",", index_col=0, encoding='latin1')
    print("Mapping has shape: " + str(mapping.shape))

    map_filt = mapping.loc[mapping['empo_3'] == "Water (non-saline)"]
    print("After selecting for biome, mapping has shape " + str(map_filt.shape))

    bools = [i in otu.columns.values for i in map_filt.index.values]
    map_filt = map_filt.loc[bools]
    print("After selecting just the samples present in the otu table: " + str(map_filt.shape))

    otu_sort = otu.reindex(sorted(otu.columns.values), axis = 1)
    map_filt_sort = map_filt.reindex(sorted(map_filt.index.values), axis = 0)
    nsamples = map_filt.shape[0]
    bools_correct = [map_filt_sort.index.values[i] == otu_sort.columns.values[i] for i in range(nsamples)]
    print("After rearranging, we have " + str(np.sum(bools_correct)) + " matching samples")
    if (np.sum(bools_correct) == map_filt_sort.shape[0]) and (np.sum(bools_correct) == otu_sort.shape[1]):
        print("Safe to continue")
    else:
        print("STOP! Something is wrong.")
        
        
    #temperature
    #phosphate
    #ammonia
    #map_filt_sort.loc[map_filt_sort['temperature_deg_c']]
    bools =  ~map_filt_sort['temperature_deg_c'].isnull() 
    map_temp = map_filt_sort.loc[~map_filt_sort['temperature_deg_c'].isnull()]
    otu_temp = otu_sort.loc[:, bools]
    bools_correct = [map_temp.index.values[i] == otu_temp.columns.values[i] for i in range(map_temp.shape[0])]
    print("We will be working with " + str(np.sum(bools_correct)) + " samples that have temperature information")
    
    
    #One final check after all transformations
    qual_vecs = qual_vecs_sort
    bools_correct = [qual_vecs.index.values[i] == otu_temp.index.values[i] for i in range(ntaxa)]
    if (np.sum(bools_correct) == qual_vecs_sort.shape[0]) and (np.sum(bools_correct) == otu.shape[0]):
        print("Safe to continue")
    else:
        print("STOP! Something is wrong.")
        
    pd_qual_vecs = pd.DataFrame(qual_vecs)
    otu = otu_temp.T
    
    otu_train = otu.loc[map_temp.study_id != 1883, :]
    otu_test = otu.loc[map_temp.study_id == 1883, :]
    map_train = map_temp.loc[map_temp.study_id != 1883, :]
    map_test = map_temp.loc[map_temp.study_id == 1883, :]
    #map_temp = map_temp.loc[map_temp.study_id != 1041, :]
    return(otu_train, otu_test, pd_qual_vecs, map_train, map_temp)


def normalize(otu):
    #Normalize
    sample_sums = otu.sum(axis=1)
    otu_norm = otu.div(sample_sums, axis=0)
    return(otu_norm)

def biofilter(otu):
    #Filter for useful taxa
    file = open('feature_selection/taxa_lowphy_highcos.obj', 'rb')
    taxa_lowphy_highcos = pickle.load(file)
    file.close()
    otu_use = otu[list(taxa_lowphy_highcos)]
    return(otu_use)

def embed(otu, qual_vecs):
    qual_vecs_use = qual_vecs.loc[list(otu.columns.values)]
    df = pd.DataFrame(np.dot(otu, qual_vecs_use), index = otu.index.values)
    return(df)

In [ ]:
otu_train, otu_test, qual_vecs_500, map_train, map_test = getData(qual_vec_type = "500")

otu_norm = normalize(otu_train)
otu_biofilter_norm = biofilter(otu_norm) #Biofilter using new embeddings
otu_emb_norm_500 = embed(otu_norm, qual_vecs_500)

In [ ]:
kf = KFold(n_splits = 3)
mses = list()
features_list = list()
for train_i, test_i in kf.split(otu_norm):
    X_train, X_test = otu_norm.iloc[train_i,:], otu_norm.iloc[test_i, :]
    y_train, y_test = map_train.iloc[train_i, :], map_train.iloc[test_i, :]
    y_train = y_train.temperature_deg_c
    y_test = y_test.temperature_deg_c
    
    print(X_train.shape)
    print(y_train.shape)
    k_obj = SelectKBest(mutual_info_regression, k= 10).fit(X_train, y_train)
    features = otu_norm.columns.values[k_obj.get_support()]
    
    
    print("features selected")
    m = svm.SVR(gamma = "scale")
    m.fit(X_train[features], y_train)
    print("Model fit")
    
    preds = m.predict(X_test[features])
    mse = np.mean([np.square(y_test[i] - preds[i]) for i in range(len(preds))])
    mses.append(mse)
    features_list.append(features)

In [ ]:
intersection = set(features_list[0])
for i in range(1, len(features_list)):
    intersection = intersection.intersection(set(features_list[i]))
    
intersection
file = open("feature_selection/ml_filtered_features.obj", "wb")
pickle.dump(intersection, file)
file.close()

file = open("feature_selection/ml_filtered_cv_mses.obj", "wb")
pickle.dump(mses, file)
file.close()